In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.formula.api import ols
import sklearn as sk 
from sklearn.model_selection import train_test_split

In [12]:
import pandas as pd
from scipy.stats import ks_2samp, chi2_contingency
from sklearn.model_selection import cross_validate, ShuffleSplit
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

#AMÉRICA DO SUL

data = pd.read_excel("América do Sul_MODEL.xlsx", sheet_name='Sheet1')

cat_col_minus_oh = ["sex", "student_accommodation", "work", 
                    "income_grupos de referência pelo percentil_20,40,60,80,100",
                    "sedentary_behavior", 'sedentary_2']

one_hotted = ["marital_status", "gender_identity", "sexual_orientation", "country_code"]                
target_col = "gad7_class"

data = data.drop(columns=["continent_code"])

X = data.drop(columns=[target_col])
y = data[target_col]

df = pd.DataFrame(data)
df.to_excel("América do Sul_MODEL_treated.xlsx", index=False)
df.describe()

,country_code,age,sex,gender_identity,sexual_orientation,bmi,marital_status,student_accommodation,work,"income_grupos de referência pelo percentil_20,40,60,80,100",score_food_smile,score_subs_smile,score_Physical Activity_smile,score_stress_smile,score_social_smile,score_sleep_smile,score_envir_smile,score_total_smile,sedentary_behavior,sedentary_2,gad7_class
count,7377.000000,7377.000000,7366.000000,7318.000000,7343.000000,7029.000000,7358.000000,7364.000000,7353.000000,6622.000000,7369.000000,7307.000000,7330.000000,7343.000000,7329.000000,7342.000000,7342.000000,7114.000000,7370.000000,7373.000000,7377.000000
mean,0.660024,19.779992,0.397231,0.034299,0.633256,23.386815,1.081136,0.979223,0.806610,1.959680,7.571584,14.865608,7.366712,3.880567,12.844999,9.485835,8.605148,64.640146,2.034600,1.119626,0.449641
std,1.299056,3.087655,0.489358,0.182009,1.048884,4.493129,0.538142,0.142646,0.394983,1.418598,1.590574,1.610601,2.548647,1.546530,4.821286,4.571043,2.096251,7.711449,1.015164,1.079484,0.497491
min,0.000000,16.000000,0.000000,0.000000,0.000000,12.818459,0.000000,0.000000,0.000000,0.000000,3.000000,4.000000,3.000000,2.000000,3.000000,3.000000,4.000000,34.000000,0.000000,0.000000,0.000000
25%,0.000000,18.000000,0.000000,0.000000,0.000000,20.312500,1.000000,1.000000,1.000000,1.000000,6.000000,14.000000,5.000000,3.000000,9.000000,6.000000,7.000000,59.000000,1.000000,0.000000,0.000000
50%,0.000000,19.000000,0.000000,0.000000,0.000000,22.666666,1.000000,1.000000,1.000000,2.000000,7.000000,15.000000,7.000000,4.000000,14.000000,8.000000,9.000000,65.000000,2.000000,1.000000,0.000000
75%,1.000000,20.000000,1.000000,0.000000,1.000000,25.461981,1.000000,1.000000,1.000000,3.000000,9.000000,16.000000,9.000000,5.000000,17.000000,12.000000,10.000000,70.000000,3.000000,2.000000,1.000000
max,5.000000,35.000000,1.000000,1.000000,4.000000,65.000000,4.000000,1.000000,1.000000,4.000000,12.000000,16.000000,12.000000,8.000000,20.000000,20.000000,16.000000,93.000000,4.000000,4.000000,1.000000


In [13]:
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Assuming X and y are already defined, as well as complete_data_idx and missing_data_idx.

# Define feature lists
one_hotted = ["marital_status", "gender_identity", "sexual_orientation", "country_code", 
              "sex", "student_accommodation", "work"]

num = ['age', 'bmi', 'score_food_smile', 'score_subs_smile', 'score_Physical Activity_smile',
       'score_stress_smile', 'score_social_smile', 'score_sleep_smile',
       'score_envir_smile', 'sedentary_behavior', 'sedentary_2',
       'income_grupos de referência pelo percentil_20,40,60,80,100']

# Create the preprocessor for one-hot encoding and scaling
onehot_transformer = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
scaler_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("onehot", onehot_transformer, one_hotted),
        ("scaler", scaler_transformer, num),
    ],
    remainder="passthrough"
)

X_transformed = preprocessor.fit_transform(X)

onehot_names = preprocessor.named_transformers_["onehot"].get_feature_names_out(one_hotted)
scaled_names = num
remaining = [col for col in X.columns if col not in one_hotted + num]
all_feature_names = np.concatenate([onehot_names, scaled_names, remaining])

X_transformed_df = pd.DataFrame(X_transformed, index=X.index, columns=all_feature_names)

missing_data_idx = X_transformed_df[X_transformed_df.isna().any(axis=1)].index.to_numpy()
complete_data_idx = X_transformed_df.dropna().index.to_numpy()

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
splits = []

for train_pos, test_pos in cv.split(complete_data_idx):
    train_idx = complete_data_idx[train_pos]
    test_idx = complete_data_idx[test_pos]
    
    train_idx = pd.Index(train_idx).union(missing_data_idx)
    
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    splits.append({
        "X_train": X_train,
        "X_test": X_test,
        "y_train": y_train,
        "y_test": y_test
    })

inputed_splits = []
knn_imputer = KNNImputer(n_neighbors=1)
for split in splits:
    X_train_inputed = pd.DataFrame(
        knn_imputer.fit_transform(split["X_train"]),
        columns=split["X_train"].columns,
        index=split["X_train"].index
    )
    inputed_splits.append(X_train_inputed)

# Debugging: check shapes of one of the splits
print(f"Treino: {splits[0]['X_train'].shape}, Teste: {splits[0]['X_test'].shape}")
inputed_splits[0].describe()

Treino: (6149, 20), Teste: (1228, 20)


,country_code,age,sex,gender_identity,sexual_orientation,bmi,marital_status,student_accommodation,work,"income_grupos de referência pelo percentil_20,40,60,80,100",score_food_smile,score_subs_smile,score_Physical Activity_smile,score_stress_smile,score_social_smile,score_sleep_smile,score_envir_smile,score_total_smile,sedentary_behavior,sedentary_2
count,6149.000000,6149.000000,6149.000000,6149.000000,6149.000000,6149.000000,6149.000000,6149.000000,6149.000000,6149.000000,6149.000000,6149.000000,6149.000000,6149.000000,6149.000000,6149.000000,6149.000000,6149.000000,6149.000000,6149.000000
mean,0.677834,19.787282,0.395349,0.034152,0.635063,23.357594,1.079525,0.978858,0.808912,1.966499,7.571800,14.868109,7.351114,3.891364,12.840787,9.506749,8.618312,64.645959,2.038380,1.120507
std,1.330709,3.097362,0.488965,0.181634,1.049916,4.495082,0.534453,0.143868,0.393190,1.420359,1.593208,1.606147,2.554257,1.549205,4.809364,4.601790,2.104352,7.761681,1.012523,1.083359
min,0.000000,16.000000,0.000000,0.000000,0.000000,12.818459,0.000000,0.000000,0.000000,0.000000,3.000000,4.000000,3.000000,2.000000,3.000000,3.000000,4.000000,34.000000,0.000000,0.000000
25%,0.000000,18.000000,0.000000,0.000000,0.000000,20.281233,1.000000,1.000000,1.000000,1.000000,6.000000,14.000000,5.000000,3.000000,9.000000,6.000000,7.000000,59.000000,1.000000,0.000000
50%,0.000000,19.000000,0.000000,0.000000,0.000000,22.656250,1.000000,1.000000,1.000000,2.000000,7.000000,15.000000,7.000000,4.000000,14.000000,8.000000,9.000000,65.000000,2.000000,1.000000
75%,1.000000,20.000000,1.000000,0.000000,1.000000,25.393431,1.000000,1.000000,1.000000,3.000000,9.000000,16.000000,9.000000,5.000000,17.000000,12.000000,10.000000,70.000000,3.000000,2.000000
max,5.000000,35.000000,1.000000,1.000000,4.000000,60.000000,4.000000,1.000000,1.000000,4.000000,12.000000,16.000000,12.000000,8.000000,20.000000,20.000000,16.000000,93.000000,4.000000,4.000000
